## Messung der OCR-Qualität in Python

In this notebook we evaluate OCR quality. We'll use the same sample article for evaluation:


![grippe.png](grippe.png)

### Imports and OCR

Let us OCR it first:

In [2]:
import pytesseract
from PIL import Image

In [7]:
ocr_output = pytesseract.image_to_string(Image.open('grippe.jpeg'), lang='frk')  # using German fraktur OCR model

In [8]:
print(ocr_output)

Zie Grippe wüfel weiter

Zunahme der f<weren Fälle in Berlin.

Die Zahl der Grippefälle iſt in den leßten
beider Tagen auch in Groß-Berlin noH
erf>liS zeitiegen. Die Worenhäuſer und ſon-
Haen aroßen GeſhHäfte, die Krirgs- unh die prie
n Betriebe lagen, daß übermäig viele An«

* fich hcben rex? melden müſſen,-und an<
; “ Loft und 5ei der Straßenbahn iſt der
ſoz der Grippelranken bedeuten) g&

MeB R 2 8 1



### Manually create  the 'ground truth' to evaluate against

In [9]:
ground_truth = input('Please insert corrected string: ')

Please insert corrected string: Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.


In [10]:
print(ground_truth)

Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.


### Calculate precision, recall, and F1-measure

In [11]:
from aux.measure_ocr_quality import measure_ocr_quality

In [12]:
precision, recall, f_score = measure_ocr_quality(ocr_output, ground_truth)

In [13]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')

Precision: 0.7757
Recall: 0.7985
F1-score: 0.7869
